# SGS models

The filtered, incompressible Navier-Stokes eq. (buoyancy, viscous and Coriolis forces are neglected):

$$
\begin{equation}
\frac{\partial \tilde{u}_i}{\partial t} + \frac{\partial \tilde{u}_i \tilde{u}_j}{\partial x_j} = - \frac{\partial \tilde{p}^*}{\partial x_i}  - \frac{\partial \tau_{ij}^d} {\partial x_j} + f_i ,
\end{equation}
$$

where $f_i$ are body forces (e.g. AD forces, half-channel driving pressure, etc.) and

$$
\tilde{p}^* \equiv  \frac{\tilde{p}}{\rho}+ \frac{1}{3} \tau_{kk}
$$


$$
\begin{align}
        \tau_{ij}^d \equiv \tau_{ij} - \frac{1}{3} \tau_{kk} \delta_{ij} 
\end{align}
$$

In this formulation with the "modified, kinematic pressure", $\tilde{p}^*$, we only need to obtain the "deviatoric SGS tensor", $\tau_{ij}^d$, which is typically modelled as: 

$$
\begin{align}
        \tau_{ij}^d &= - 2 \nu_{sgs} \tilde{S}_{ij} ,
\end{align}
$$

where $\tilde{S}$ is the symmetric, resolved, strain-rate tensor. The question is now just, how do we model $\nu_{sgs}$?




### Smagorinsky model

The classic Smagorinsky model developed by Smagorinsky (1963), Lilly (1967) and deployed in the first ever LES by Deardorff (1970).

$$
\begin{align}
    \nu_{sgs} &= (C_S \Delta)^{2} \mathcal{S}  ,
\end{align}
$$

where $\Delta$ is filter width (in finite-volume $\Delta = (\Delta_x \Delta_y \Delta_z)^{1/3}$) and $\mathcal{S} \equiv \sqrt{2 \tilde{S}_{ij} \tilde{S}_{ij}}$ is a charecteristic inverse time scale. For isotropic flow $C_S = 0.16$, while in wall-bounded atmospheric flows "Mason wall damping" (Mason and Thompson 1992) is typically applied:

$$
\frac{1}{(C_S \Delta)^{2}} = \frac{1}{(0.16 \Delta)^{2}} + \frac{1}{\left[\kappa (z + z_0)\right]^{2}} ,
$$

i.e. for large $z$, the isotropic $C_S$ is used, whille close to the wall, the law-of-wall is used.

- Simple model, but requires tuning of $C_S$.

### Bardina model (aka. similarity model)

Developed by Bardina (1980). The model does *not* use any $\nu_{sgs}$, but models the Reynolds stress directly as:

$$
\tau_{ij} = \overline{\tilde{u}_i \tilde{u}_j} - \tilde{u}_i \tilde{u}_j
$$

The overbar is another filtering operation (Bardina choose same filtering width as original field, $\Delta$). The RHS is also known as the Leonard tensor (see Dynamic Smagorinsky, where they however use $2 \Delta$ for the extra filtering). The Bardina model is known as a "scale-similarity"-model, because it is derived by assuming that the turbulence behaves the same just above and below the cutoff length.

The Bardina model is known to not be dissipative enough, so Zang (1993) included a Smagorinsky-like term to fix this:

$$
\tau_{ij} = \overline{\tilde{u}_i \tilde{u}_j} - \tilde{u}_i \tilde{u}_j -  2 (C_S \Delta)^2 \mathcal{S} \tilde{S}_{ij} 
$$

This is called a "mixed-scale"-model.

- Note, that some authors write the Bardina model slightly different, because they refer to the deviatoric stress tensor, $\tau_{ij}^d$, and not the full stress tensor, $\tau_{ij}$.



### Deardorff model

Developed by Deardorff (1980) and described by Moeng (1984).

$$
\nu_{sgs} = C_k \ell_{LES}  k_{sgs}^{1/2} ,
$$

where $\ell_{LES} = \Delta$ (for neutral). $k_{sgs}$ is obtained from a transport equation:


$$
\begin{equation}
\frac{D k_{sgs}}{D t} = \mathcal{P} + \mathcal{D} + \mathcal{B} - \varepsilon
\end{equation}
$$        




The expressions for the different terms on the right hand side can be found in for example Sullivan (1994) or Moeng (2015).

- Several tuning parameters ($C_k$, $C_\epsilon$ and $\ell_{LES}$ in non-neutral conditions)
- Mahdi says this SGS model is known to have grid convergence problems, e.g. Sullivan (2016).

### Dyanmic Smagorinsky model

Developed by Germano (1991). Same as Smagorinsky, but dynamically adjusts $C_S$ (no need to manually specify $C_S$-value!!).

For each time step and each point in the flow, it is calculated as:

$$
C_S^2 = \frac{\langle L_{ij} M_{ij}\rangle }{\langle M_{ij} M_{ij}\rangle}
$$

- $L_{ij} \equiv \overline{\tilde{u}_i \tilde{u}_j} - \overline{\tilde{u}_i}~\overline{\tilde{u}_j}$ is the "Leonard tensor", which can be obtained from the normal "$\Delta$-resolved" flow field by using a test filter with $2 \Delta$ (denoted by overbar).
- $M_{ij} \equiv 2 \Delta^2 \left( \overline{\mathcal{S} \tilde{S}_{ij}} - 4 \overline{\mathcal{S}}~\overline{ S_{ij}} \right)$. Not sure if this tensor has a name?  
- The brackets $\langle \rangle$ means somekind of average. In flat terrain, horizontal average is typically used, while in complex terrain a Lagrangian average (Meneveau 1996) is typically used. This average is needed for numerical stability (see Porte-Agel von Karman lecture note). Sarlak (2015) used a 3D box average (27-point stencil) in EllipSys. 
- Does not perform too well for heterogeneous terrain, e.g. roughness changes.
- No need for wall damping function.
- No tuning parameters.
- Scale invariance is assumed, i.e. $C_S$ is the same for the $\Delta$-resolved flow and $2 \Delta$-resolved flow. This is not necessary true and is corrected in Porte-Agel's 2000 "scale-dependent"-model.

### Lagrangian model

Developed by Meneveau (1996). Same as Dynamic Smagorinsky, but the averages $\langle \rangle$ are done over pathlines ("Lagrangian average") instead, which means that the model can also be applied in complex terrain or in other inhomogeneous conditions (e.g. wind farm!).

<img src="lagrangian_average.png"/>

In the Langrangian average more weight is put on the most recent positions of the fluid parcel ("expoential decreasing memory"), which is represented by the grayscale color in the above figure. Hence, only a few time steps back are needed. In principle two transport equations are needed to obtain the Lagrangian avergage, but this is expensive, so Meneveau developed a simple algebraic model, which only introduces 10% CPU overhead compared to the original Dynamic Smagorinsky model.

- Same as Dynamic Smagorinsky, but can also be used in complex terrain.
- Can be used in both homo- and hetereogeneous flat terrain.
- Only 10% more expensive.
- Assumes scale invariance, like the Dynamic Smagorinsky model.

### WALE model

The Wall-Adpating Local Eddy viscosity by Nicoud and Ducros (1999) does $\underline{not}$ use Smagorinsky as basis, but rather model $\nu_{sgs}$ as:

$$
\nu_{sgs} = \left( C_w \Delta \right)^2 \frac{\left(\tilde{S}_{ij}^d \tilde{S}_{ij}^d \right)^{3/2}}{\left(\tilde{S}_{ij} \tilde{S}_{ij} \right)^{5/2} + \left(\tilde{S}_{ij}^d \tilde{S}_{ij}^d \right)^{5/4}} ,
$$

where

$$
\tilde{S}_{ij}^d \equiv \frac{1}{2} \left(\tilde{g}_{ij}^2 + \tilde{g}_{ji}^2 \right) - \frac{1}{3} \tilde{g}_{kk}^2 \delta_{ij}
$$

$$
\tilde{g}_{ij} \equiv \frac{\partial \tilde{u}_i}{\partial x_j} .
$$

- While Smagorinsky based models are based on $\mathcal{S}$ (i.e. only depends on the symmetric part of the velocity gradient), WALE also uses some of the anti-symmetric part (not explicitly shown in the above equation, though).
- No wall damping needed.
- Has one tuning parameter, $C_w$.
- Not sure, if it has been tested for atmospheric flows.


### Scale-depedent model

Developed by Porte-Agel (2000). Same as Dynamic Smagorinsky, but fixes the scale invariance problem. 


$$
C_S^2 = \frac{\langle L_{ij} M_{ij}\rangle }{\langle M_{ij} M_{ij}\rangle} , 
$$

where $L_{ij}$ is the same as in the Dynamic Smagorinsky model, but $M_{ij}$ is different:

$$
M_{ij} \equiv 2 \Delta^2 \left( \overline{\mathcal{S} \tilde{S}_{ij}} - 4 \beta \overline{\mathcal{S}}~\overline{ S_{ij}} \right).
$$

Note, the special case $\beta = 1$ corresponds to the Dynamic Smagorinsky model. For the scale-dependent model, $\beta$ is solved from the fifth order polynomial (only one physical solution of this):

$$
A_0 + A_1 \beta + A_2 \beta^2 + A_3 \beta^3 + A_4 \beta^4 + A_5 \beta^5 = 0 ,
$$

where the coefficients are obtained from expressions using both "$2 \Delta$-resolved velocity" and "$4 \Delta$-resolved velocity" components, i.e. a second test filter with $4 \Delta$ is used, see Porte-Agel for all the expressions.

- Fixes the scale invariance problem of the Dynamic Smagorinsky model $\rightarrow$ more realistic mean profiles and statistics.
- Almost no CPU overhead compared to the Dynamic Smagorinsky model.
- Only applicable to flat terrain, but Bou-Zeid (2005) extended the model to complex terrain using Lagrangian averages.

### Lagrangian scale-depedent model

Developed by Bou-Zeid (2005). It is simply a combination of the Lagrangian model of Meneveau (1996) and the scale-dependent model of Porte-Agel (2000).

- Can be applied in both flat and complex terrain (even with hetereogeneous roughness).
- No tuning parameters.
- Most sophisticated SGS model according to Mahdi (along with AMD).


### Mixed scale model

Developed by Ta Phuoc (1994) and Sagaut (1995). A short review is given by Sagaut (2001), where it is called a "mixed scaled model" (not sure what the connection to Zang's mixed scale model is???). Applied in Troldborg (2007) as:

$$
\begin{align}
            \nu_{sgs} &= c_{mix} l_{mix}^{3/2} |\nabla \times \vec{u}|^{1/2} k_{sgs}^{1/4}
\end{align}
$$

with $c_{mix} = 0.1$, $l_{mix} =  \Delta$, $\Delta = \delta_{scale} \cdot (\Delta_x \Delta_y \Delta_z)^{1/3}$ and $k_{sgs} = \frac{1}{2} \sum_{i=1}^{3}(\tilde{u}_i - \langle \tilde{u}_i \rangle_{loc})^2 $, where $\langle \rangle_{loc}$ is local average of the surrounding cells. The default value of $\delta_{scale}=1$ can be changed with the $\texttt{delta_scale}$ input parameter.

- Has the tuning parameter, $C_{mix}$.
- Not sure what this model is good for...
- Sarlak (2016) also tested this model with $|\mathcal{S}|$ instead of $|\Omega| = |\nabla \times \vec{u}|$.

### Modulated gradient model

Developed by Lu (2010). It is *not* a Smagorinsky-type model, but instead a *gradient*-type model (already considered in the 70's), which is based on the Taylor expansion of $\tau_{ij}$:

$$
\tau_{ij} \approx \tilde{G}_{ij} + O(\Delta^4) .
$$

Lu (2010) scales the above equation with $k_{sgs}$ and uses an anisotopic formulation of $\tilde{G}_{ij}$:

$$
\tau_{ij} = 2 k_{sgs} \left( \frac{\tilde{G}_{ij}}{\tilde{G}_{kk}} \right)
$$

$$
\tilde{G}_{ij} = \left( \frac{\Delta_{x_k}}{12} \right) \left( \frac{\partial \tilde{u}_i}{\partial x_k} \frac{\partial \tilde{u}_j}{\partial x_k}\right)
$$

$$
k_{sgs} = \mathcal{H}(P) \frac{4 \Delta^2}{C_\varepsilon^2} \left( - \frac{\tilde{G}_{ij}}{\tilde{G}_{kk}} \tilde{S}_{ij} \right)^2 .
$$

The latter equation is derived using local equilibrium hypothesis, i.e. $\mathcal{P} = \varepsilon$, and clipping with a heaviside function, $\mathcal{H}$, is applied for numerical stability.

- There is a nice von Karman lecture note by Porte-Agel about the model.
- There is one tuning parameter, $C_\varepsilon$. It was set to a constant (=1) in Lu (2010), while Lu (2014) implemented a dynamic procedure to obtain it.



### Anisotropic Minimum Dissipation (AMD) model

Developed by Rozema (2015) based on the isotropic Minimum Disipation model (aka. QR-model) of Verstappen (2010). Both of these papers are pretty complicated, while Abkar (2016) and Abkar (2017) are more readable.

$$
\nu_{sgs} = -\frac{\left(\Delta_k \sqrt{C_k} \frac{\partial \tilde{u}_i}{\partial x_k} \right) \left(\Delta_k \sqrt{C_k} \frac{\partial \tilde{u}_j}{\partial x_k} \right)\tilde{S}_{ij}}{\frac{\partial \tilde{u}_m}{\partial x_l} \frac{\partial \tilde{u}_m}{\partial x_l}} ,
$$

where $C_k = 1/3$ for second-order schemes (used in EllipSys), while  $C_k = 1/12$ for spectral methods (the two horizontal directions in Mahdi's code).

- Note, some papers (Abkar 2016) has $\sqrt{C_k} \rightarrow C_k$ and then $C_k = 1 / \sqrt{3}$ and $C_k = 1 / \sqrt{12}$ are therefore used instead.
- Mahdi said that this (together with scale-depedent Langrangian) is the most sophisticated SGS model.


# SGS models in EllipSys

- Standard Smagorinsky
- Ta Phuoc mixed scale model
- Deardorff
- AMD

Hamid implemented Dynamic Smagorinsky in his thesis, but I think he has it in a local branch..


# A-priori test of SGS models

Using DNS data, one can test how well a SGS model performs without running any LES! 

Going all the way back to the filtered NS-equation, the SGS stress is defined:

$$
\tau_{ij} \equiv \widetilde{u_i u_j} - \tilde{u}_i \tilde{u}_j
$$

Both RHS terms can be calculated by applying a filter to the DNS data. This is then used as the "reference" SGS stress, which can be compared the modelled SGS stress, e.g. we could try to calculate $\tau_{ij}$ using the Smagorinsky model (one can also obtain $\tilde{S}_{ij}$ from the DNS data).




In [1]:
# Maybe make a-priori test of SGS models using some DNS data? 
# I don't think there are any DNS of atmospheric flows though....

import numpy as np
import xarray
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.pyplot import cm


# PLOT STYLE ##################################
mpl.style.use('classic')
plt.rcParams['font.family'] = 'STIXGeneral'
plt.rcParams["legend.scatterpoints"] = 1
plt.rcParams["legend.numpoints"] = 1
plt.rcParams['grid.linestyle'] = ':'
mpl.rcParams['lines.linewidth'] = 2
plt.rcParams['axes.grid']=True
yd = dict(rotation=0, ha='right')
plt.close('all')

